# Calculate Diffusion Coefficient

The Free Energy Profile F(q), for radsorbate hopping along the reaction coordinate can be calculated by the mean energy of insertion of the (spherical) adsorbate molecule using the FF parameters in the planes orthogonal to the reaction coordinate:
$$ F(q) = -k_BT ln\langle e^{-\beta \Delta U} \rangle_q $$
Here the brackets denote averaging the Boltzmann factor ocer the square grids of resolution dx Å resolution.

The 1-dim cage-to-cage hopping rate is given by: 
$$ k_{C_1 \rightarrow C_2} = \kappa \sqrt{\frac{k_B T}{2 \pi m}} \frac{e^{-\beta F(q^*)}}{\int_{cage} e^{-\beta F(q)} \,dq} $$
where m is the mass of the adsorbate, q is the reaction coordinate, F is the Free Energy as a function of the reaction coordinate, T is the temperature (T = 298 K), and $\kappa$ is the Bennett-Chandler dynamic correction. $\kappa$ = 1 is a good approximatin for infinite dilution. 
The space is partitioned such that the dividing surface is perpendicular to the reactin coordinate and passes through the location of the maximum free energy barrier along the path F(q*). 


To get the self-diffusion coefficient ($D_s$) 
$$ D_s = \frac{\kappa}{2d} \lambda^2 k_{C_1 \rightarrow C_2} $$

$\lambda$ is the cage-center to cage-center lattice distance.

In [1]:
using PorousMaterials
using Statistics
using PyPlot
using Cubature
using Printf 

┌ Warning: Error requiring `OffsetArrays` from `ArrayInterface`
│   exception = (UndefVarError(:IdOffsetRange), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007fbe00bdfcef, Ptr{Nothing} @0x00007fbe00c72d24, Ptr{Nothing} @0x00007fbdf29fdda2, Ptr{Nothing} @0x00007fbe00c55769, Ptr{Nothing} @0x00007fbe00c71f15, Ptr{Nothing} @0x00007fbe00c71bce, Ptr{Nothing} @0x00007fbe00c72811, Ptr{Nothing} @0x00007fbe00c73297, Base.InterpreterIP in top-level CodeInfo for ArrayInterface at statement 11, Ptr{Nothing} @0x00007fbe00c8eb31, Ptr{Nothing} @0x00007fbe00c90949, Ptr{Nothing} @0x00007fbda19292b1, Ptr{Nothing} @0x00007fbda19292dc, Ptr{Nothing} @0x00007fbe00c55769, Ptr{Nothing} @0x00007fbe00c71f15, Ptr{Nothing} @0x00007fbe00c71bce, Ptr{Nothing} @0x00007fbe00c72811, Ptr{Nothing} @0x00007fbe00c72b90, Ptr{Nothing} @0x00007fbe00c7304a, Base.InterpreterIP in MethodInstance for err(::Any, ::Module, ::String) at statement 2, Ptr{Nothing} @0x00007fbda1929227, Ptr{Nothing} @0x00007fbda192923c, Ptr

In [2]:
# system setup
const R = 8.31446261815324 / 1000 # Ideal Gas Constant, units: kJ/(mol-K)
temp = 298.0 # units: K
β = 1 / (R * temp) # units: (kJ/mol)⁻¹

ljff = LJForceField("UFF") # r_cut = 14.0 Å, mixing_rule = Lorentz-Berthelot


adsorbates = Dict(:Xe => Dict(:molecule => Molecule("Xe"),
                              :mol_mass => 131.293 / 1000), # kg/mol
                  :Kr => Dict(:molecule => Molecule("Kr"), 
                              :mol_mass => 83.798 / 1000))  # kg/mol

xtals = Dict(:nipyc   => Crystal("NiPyC2_experiment.cif"),
             :nipycnh => Crystal("Pn_Ni-PyC-NH2.cif"));

┌ Info: Crystal NiPyC2_experiment.cif has Pn space group. I am converting it to P1 symmetry.
│         To prevent this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ Xtals /home/ng/.julia/packages/Xtals/DSCSR/src/crystal.jl:433
┌ Info: Crystal Pn_Ni-PyC-NH2.cif has Pn space group. I am converting it to P1 symmetry.
│         To prevent this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ Xtals /home/ng/.julia/packages/Xtals/DSCSR/src/crystal.jl:433


# Monte Carlo Integration TST Method 
1. Determine integration bounds: 
    We will integrate over the entire unit cell, but only points that are 
    accessible will contribute to the integral. Inaccessible points will be rejected. 
    1. calculate grid = energy_grid()
    2. get seg_grid = PorousMaterials._segment_grid()
    3. get acc_grid = calculate_accessibility_grid
    4. get fractional coordinates of molecule (e.g. xf = rand(3))
    5. check that we are in the channel that we want AND the gridpoint is accessible
        - IF seg_grid.data(xf_to_id(xtal.box, xf) == channel_ID && accessible(acc_grid, xf)
             THEN perform calculation vdw_energy(xf)
          ELSE reject point (return exp() = 0? or just return nothing?)
2. Calculate average Boltzmann Factor of the Free energy 


**Note:** for MC integration, the cross-sectional area of the channel at the slice we are evaluating is given as the area of the integrating domain multiplied by the ratio points inside the channel to the total number of MC samples on that domain, i.e. $$ A_{xc} = A_{domain} *\left ( \frac{N_{inside}}{N_{total}} \right ) $$

In [20]:
grid_resolution = 0.1 # units: Å
energy_cutoff = 10.0 # units: kJ/mol

grid = energy_grid(xtals[:nipycnh], adsorbates[:Xe][:molecule], ljff, resolution=grid_resolution)

segmented_grid = PorousMaterials._segment_grid(grid, energy_cutoff, true)

Computing energy grid of Xe in Pn_Ni-PyC-NH2.cif
	Regular grid (in fractional space) of 108 by 120 by 72 points superimposed over the unit cell.
Found 3 segments


Regular grid of 108 by 120 by 72 points superimposed over a unit cell and associated data.
	units of data attribute: Segment_No
	origin: [0.000000, 0.000000, 0.000000]


In [23]:
# so the fact that there is a connection between channels makes things somewhat more complicated
#         Noted seg. 3 --> 1 connection in (1, 0, 0) direction
# the nice thing is that it is in the (1, 0, 0) direction while 
# the other channels are in the (0, 0, 1) direction. 
# The reason this makes it easier is that Ds = (Dx + Dy + Dz) / 3  (see refs.)
# meaning that I can do the calculations for each channel independently, 
# disregarding the channels that are not accessible to each other 
# (this is to say that a molecule in channel 1 cannot diffuse into channel 2)
# note: channel 2 is essentially identical to channel 1, bit it is staggered (see Egrid figs in paper)
# could be useful to compare the diffusion in channel 2 to channel 1 (maybe take the average of the two?)

# Oh, wait, but Noted seg. 3 --> 3 connection in (0, 0, 1) direction.
# so, I'm not sure what this channel is... maybe there is another partial channel 
# present in the unit cell of nipycnh..?
energy_cutoff = 3.0 # units: kJ/mol

acc_grid, nb_segs_blocked, porosity = compute_accessibility_grid(xtals[:nipycnh], 
                                                                 adsorbates[:Xe][:molecule], 
                                                                 ljff;
                                                                 resolution=grid_resolution,
                                                                 energy_tol=energy_cutoff,  
                                                                 energy_units=:kJ_mol,
                                                                 verbose=true, 
                                                                 write_b4_after_grids=false,
                                                                 block_inaccessible_pockets=true)

Computing accessibility grid of Pn_Ni-PyC-NH2.cif using 3.000000 kJ_mol potential energy tol and Xe probe...
Computing energy grid of Xe in Pn_Ni-PyC-NH2.cif
	Regular grid (in fractional space) of 108 by 120 by 72 points superimposed over the unit cell.
Found 3 segments
Noted seg. 1 --> 1 connection in (0, 0, 1) direction.
Noted seg. 2 --> 2 connection in (0, 0, 1) direction.
Noted seg. 3 --> 3 connection in (0, 0, 1) direction.
Noted seg. 3 --> 1 connection in (1, 0, 0) direction.
	Found 7 simple cycles in segment connectivity graph.
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
Segment 1 classified as accessible channel.
Segment 2 classified as accessible channel.
Segment 3 classified as accessible channel.


(Regular grid of 108 by 120 by 72 points superimposed over a unit cell and associated data.
	units of data attribute: accessibility
	origin: [0.000000, 0.000000, 0.000000]
, 0, Dict(:b4_blocking => 0.055514831961591224, :after_blocking => 0.055514831961591224))

In [22]:
# get list of connections present in segmented grid
# connections.str       start
# connections.dst       destination
# connections.direction direction
connections = PorousMaterials._build_list_of_connections(segmented_grid)

Noted seg. 1 --> 1 connection in (0, 0, 1) direction.
Noted seg. 2 --> 2 connection in (0, 0, 1) direction.
Noted seg. 3 --> 3 connection in (0, 0, 1) direction.
Noted seg. 3 --> 1 connection in (1, 0, 0) direction.


8-element Vector{PorousMaterials.SegmentConnection}:
 PorousMaterials.SegmentConnection(1, 1, (0, 0, 1))
 PorousMaterials.SegmentConnection(1, 1, (0, 0, -1))
 PorousMaterials.SegmentConnection(2, 2, (0, 0, 1))
 PorousMaterials.SegmentConnection(2, 2, (0, 0, -1))
 PorousMaterials.SegmentConnection(3, 3, (0, 0, 1))
 PorousMaterials.SegmentConnection(3, 3, (0, 0, -1))
 PorousMaterials.SegmentConnection(3, 1, (1, 0, 0))
 PorousMaterials.SegmentConnection(1, 3, (-1, 0, 0))

### Helper Functions

In [7]:
# This is the average free energy on a plane along the reaction coord. q
# this average is obtained by MC integration 
function avg_free_energy_on_plane(q::Float64, xtal::Crystal, adsorbate::Molecule,
                   seg_grid::Grid{Int64}, acc_grid::Grid{Bool}; 
                   seg_id::Int=1, dim::Int=1, nb_insertions::Union{Float64, Int64}=1e6)
    # initialize array to store energy values
    if typeof(nb_insertions) != Int64 
        nb_insertions = Int(nb_insertions)
    end
    sample_energy_on_slice = zeros(nb_insertions)
    # initialize MC counter to calculate cross-sectional area
    mc_inside = 0
    
    # make sure that xtal is replicated enough for vdW energy calculations
    rep_factors = replication_factors(xtal, ljff)
    crystal = replicate(xtal, rep_factors)
        
    # copy molecule so we don't manupulate original
    molecule = deepcopy(adsorbate)
    
    ###
    #  At this point {(0, 0, 0), (1, 1, 1)} define the bounds of the ORIGINAL crystal 
    #  in factional space
    ###
    
    # perform MC integration on a plane
    for n in 1:nb_insertions
        # get random location, in fractional coords, on the plane
        rxf_1, rxf_2 = rand(2)
        
        # NEED TO USE DIM TO FIGURE OUT WHICH POSITION IN THE ARRAY TO PLACE THE GIVEN INPUT COORDINATE
        # DO IT MANUALLY FOR NOW
        # inside ORIGINAL xtal.box
        if dim == 1
            xf = Frac([q, rxf_1, rxf_2]) 
        elseif dim == 2
            xf = Frac([rxf_1, q, rxf_2])
        elseif dim == 3
            xf = Frac([rxf_1, rxf_2, q])
        else
            @error "can only hande dim = 1, 2, or 3 currently"
        end
        
        # get voxel ID for molecule in original xtal
        xf_id = xf_to_id(seg_grid.n_pts, [xf.xf...])
        
        # calculate energy IFF we are in the desired channel and the loc is accessible
        if seg_grid.data[xf_id...] == seg_id && accessible(acc_grid, [xf.xf...])
            # incrament mc counter
            mc_inside += 1
            
            ###
            #  At this point {(0, 0, 0), (1, 1, 1)} define the bounds of the REPLICATED crystal 
            #  in factional space
            ###
            
            # redefine molecule in fractional coords w.r.t. replicated crystal
            if isa(molecule, Molecule{Cart})
                molecule = Frac(molecule, crystal.box)
            else
                translate_to!(molecule, Frac(molecule.com.xf ./ rep_factors)) # rescale Frac coords
            end
            
            # translate the probe molecule to sampling location 
            translate_to!(molecule, Frac(xf.xf ./ rep_factors))

            energy = vdw_energy(crystal, molecule, ljff) * R # units: kJ/mol
            
            push!(sample_energy_on_slice, energy)
        end
    end # MC integration
    
    ###
    #  Area from MC is just the ration of accepted samples over the total number of samples,
    #  multiplied by the area of the integrated domain
    # TODO: I know we multiply by the ratio of samples, but do we include te box side lengths?
    #       maybe check henry.jl to see how it is done there. Check Berend's book too. 
    ###
    sides = [[2, 3], [1, 3], [1, 2]]
    area_xc = getfield(xtal.box, sides[dim][1]) * getfield(xtal.box, sides[dim][2])
    area = area_xc * (mc_inside / nb_insertions)
    
    boltzmann_factors = exp.(-β * sample_energy_on_slice)
    
    avg_boltzmann_factor_on_slice = sum(boltzmann_factors) / area
    
    avg_free_energy_at_q = -log(avg_boltzmann_factor_on_slice) / β
    
    return avg_free_energy_at_q
end

avg_free_energy_on_plane (generic function with 1 method)

In [8]:
function cal_self_diffusin_coeff(xtal_key::Symbol, gas_key::Symbol, 
                                 q_str::Tuple{Any, Any},
                                 free_E::Vector{Float64}; 
                                 dim::Int=1, verbose::Bool=true)

    
    
    ###
    #  Calculate hopping rate
    ###
    # dynamic update, correction, factor, is 1 at infinite dilution
    κ = 1.0 # units: none
    
    # average velocity given by Boltzman dist.
    # conversion: 1 m/s = 10 Å/ns 
    avg_vel = 10 * sqrt((1000 * R * temperature) / (2 * π * adsorbates[gas_key][:mol_mass])) # units: Å / ns 
    # jummping frequency (hoping rate), units: (avg num successful hops) / unit time
    # hop_rate [=] ns⁻¹
    sum_over_cage = sum(exp.(-β * grid_free_E[1:q_str_id])) / q_str_xf # normalized sum, units: unitless
    
    
    hop_rate = κ * avg_vel * exp(-β * fe_q_str) / sum_over_cage
    
    ###
    #  Calculate Diffusion Coefficient
    ###
    x = [i for i in 1:grid.n_pts[dim]] * getfield(xtals[:replicated][xtal_key].box, dim) / grid.n_pts[dim]    
    λ = abs(x[grd_fe_min[2]] - x[grd_fe_min[1]]) # distance between wells, units: Å
    if verbose; println("\tDistance between wells, λ = $(λ) Å"); end
    @assert λ < getfield(xtals[:replicated][xtal_key].box, dim) "distance between wells > xtal axis"
    
    time_scale = λ / avg_vel
    if verbose; println("\tTime scale, t = $(time_scale) ns"); end
    
    diff_coeff = (κ / 6) * λ^2 * hop_rate # units: [Å²/ns]

    ### Conversion:
    # 1 Å  = 10⁻⁸ cm -> 1 Å² = 10⁻¹⁶cm²
    # 1 ns = 10⁻⁹ s
    #
    # Then,
    # 1 Å / ns = (10⁻¹⁶cm²) / (10⁻⁹ s) = 10⁻⁷ cm²/s
    ###
    conversion_factor = 1e-7
    Dₛ = diff_coeff * conversion_factor
    return Dₛ, hop_rate
end

cal_self_diffusin_coeff (generic function with 1 method)

## Perform Calculations

In [ ]:
grid_resolution = 0.1 # units: Å
energy_cutoff = 50.0 # units: kJ/mol

for xtal_key in keys(xtals)
    for gas in keys(adsorbates)
        # track runtime
        start_time = time()
        
        ###
        #  Grid calculations
        ###
        # need grid to calculate segmented grid
        grid = energy_grid(xtals[xtal_key], adsorbates[gas][:molecule], ljff, resolution=grid_resolution)
        
        # get segmented grid so we can constrain our calculation to a single channel
        segmented_grid = PorousMaterials._segment_grid(grid, energy_cutoff, true)
        
        # get accessibility grid so we know is the sample location is valid given the energy cutoff
        acc_grid, nb_segs_blocked, porosity = compute_accessibility_grid(xtal_key, 
                                                                 adsorbates[gas][:molecule], 
                                                                 ljff;
                                                                 resolution=grid_resolution,
                                                                 energy_tol=energy_cutoff,  
                                                                 energy_units=:kJ_mol,
                                                                 verbose=true, 
                                                                 write_b4_after_grids=false,
                                                                 block_inaccessible_pockets=true)
        
        
        # DETERMINE WHICH AXIS TO INTEGRATE ALONG
        q_axis = 1
        
        ###
        #  Calculate Average Free energy along reaction coordinate using MC integration
        ###
        # used for max free energy (xf, free energy)
        q_str = (nothing, nothing) 
        
        nn = 2 # increase density of slices by this factor
        step_sz = grid_resolution / (nn * getfield(xtals[:nipyc].box, q_axis))
        len = Int(ceil(1.0 / step_sz)) + 1
        x_pts = range(0.0, stop=1.0, length=len) # will be fractional coords
        
        
        avg_free_energy = Array{Float64, 1}()
        for q in x_pts 
            @assert 0.0 <= q && q <= 1.0 "invalid input, q = $(q) is outside of fractional bounds"
            # This is F(q) i.e. the average free energy as a function of the reaction coorfinate
            result = avg_free_energy_on_plane(q, xtals[:nipyc], adsorbates[:Xe][:molecule], 
                                                       segmented_grid, acc_grid,
                                                       seg_id=1, dim=q_axis, nb_insertions=1e3)
            
            push!(avg_free_energy, result)
            
            ###
            #  Track value and location (Frac) of the maximum free energy
            ###
            
            if any(isnothing.(q_str)) # not yet assigned
                q_str = (q, result)
            elseif result > q_str[2]
                q_str = (q, result)
            end
        end
        
        # runtime
        stop_time = time()
        ellapsed_time = (stop_time - start_time) / 60 # miutes
        println("runtime = $(ellapsed_time) [min]")
        
        ###
        #  Calculate Hoping Rate and Self-Diffusion Coefficient
        ###
        Dₛ, k = cal_self_diffusin_coeff(xtal_key, gas_key, q_str, free_E; 
                                 temperature::Float64=298.0, 
                                 dim::Int=1, verbose::Bool=true)
        println("\tHopping rate = $(k) [ns⁻¹]")
        println("\tSelf-diffusion Coeff. = $(Dₛ) [cm² s⁻¹] \n")
    end
end

In [11]:
### Xe in Ni(PyC)2 
# nn = 1
# 1e3 -> runtime = 0.005787030855814616 [min]
# 1e4 -> runtime = 0.03901063203811646 [min]
# 1e5 -> runtime = 0.3610572814941406 [min]
# 1e6 -> runtime = 3.740756181875865 [min]

# seem like time increases proportional to the log power increase

# nn = 2, interested to see if time doubles correspondingly
# 1e3 -> runtime = 0.013776783148447673 [min]
# 1e4 -> runtime = 0.08588219881057739 [min]
# 1e5 -> runtime = 0.8203191677729289 [min]
# 1e6 -> runtime = 7.158331453800201 [min]
###

# Integral Method

In [12]:
# function integrand(xf, xtal, adsorbate, seg_grid, acc_grid, seg_id, dim, nb_insertions)
#     avg_free_energy = avg_free_energy_on_plane(xf, xtals[:nipyc], 
#                                                adsorbates[:Xe][:molecule], 
#                                                segmented_grid, acc_grid, seg_id=1, dim=q_axis, 
#                                                nb_insertions=1e4)
    
#     result = exp(-β * avg_free_energy)
#     return result
# end






# start_time = time()

# ###
# #  perform 1-dim integration along channels
# ###
# q_axis = 1

# # bounds in fractional coordinates
# lower_bound = 0.0
# upper_bound = 1.0

# relative_tolerance = 1e-1
# absolute_tolerance = 0

# (val, err) = hquadrature(xf -> begin integrand(xf, xtals[:nipyc], 
#                                                  adsorbates[:Xe][:molecule], 
#                                                  segmented_grid, acc_grid, seg_id=1, dim=q_axis, 
#                                                  nb_insertions=1e4); end,
#                          lower_bound, upper_bound, 
#                          reltol=relative_tolerance, abstol=0, maxevals=0)

# # calc runtime
# stop_time = time()
# ellapsed_time = (stop_time - start_time) / 60 # miutes
# println("Integration Results - ")
# println("\tval = $(val)")
# println("\terr = $(err)")
# println("\tTolerances: rel_tol = $(relative_tolerance), abs_tol=$(absolute_tolerance)")
# println("runtime = $(ellapsed_time) [min]")